# ✍️ Tekst Generatie: Hoe AI tekst schrijft

Hoe genereert ChatGPT tekst? In de kern voorspelt het steeds het **volgende woord** op basis van de voorgaande woorden.

In dit notebook bouwen we zelf een heel simpel tekstgeneratiemodel om dit principe te begrijpen. We gebruiken **n-grammen**: het model kijkt naar de laatste N woorden en voorspelt het volgende.

### Wat gaan we doen?
1. Een tekst analyseren op woordpatronen
2. Een simpel model bouwen dat tekst genereert
3. Het verschil zien tussen een dom en slim model
4. Begrijpen wat "temperature" doet

---
**Instructie:** Voer iedere cel uit met **Shift+Enter**

## Stap 1: Onze trainingstekst

We gebruiken een stukje tekst als "trainingsdata". Hoe meer tekst, hoe beter het model.

In [ ]:
tekst = """
The cat sat on the mat. The cat looked at the bird. The bird sat on the tree.
The dog chased the cat. The cat ran up the tree. The bird flew away from the tree.
The dog sat on the mat. The dog looked at the cat in the tree.
The cat jumped down from the tree. The cat sat on the mat again.
The bird came back to the tree. The dog chased the bird. The bird flew over the dog.
The cat watched the dog chase the bird. The dog gave up and sat on the mat.
The cat and the dog sat on the mat together. The bird sang in the tree.
The cat listened to the bird. The dog fell asleep on the mat.
""".strip()

# Splits in woorden
woorden = tekst.lower().split()

print(f"Trainingstekst: {len(tekst)} karakters, {len(woorden)} woorden")
print(f"Unieke woorden: {len(set(woorden))}")
print(f"\nEerste 30 woorden: {' '.join(woorden[:30])}...")

## Stap 2: Woordpatronen analyseren

We kijken: na welke woorden volgen welke andere woorden? Dit noemen we **bigrammen** (paren van 2 woorden).

In [ ]:
from collections import defaultdict, Counter

# Tel voor elk woord welke woorden erna komen
bigrammen = defaultdict(Counter)

for i in range(len(woorden) - 1):
    huidig = woorden[i]
    volgend = woorden[i + 1]
    bigrammen[huidig][volgend] += 1

# Laten we een paar voorbeelden bekijken
voorbeeld_woorden = ['the', 'cat', 'dog', 'sat', 'on']

print("Na welk woord volgt wat?")
print("=" * 50)
for w in voorbeeld_woorden:
    opties = bigrammen[w].most_common(5)
    opties_str = ', '.join([f"'{v}' ({n}x)" for v, n in opties])
    print(f"  Na '{w}': {opties_str}")

## Stap 3: Tekst genereren!

Nu bouwen we een simpel model dat steeds het volgende woord kiest op basis van de waarschijnlijkheid.

In [ ]:
import random
import numpy as np

def genereer_tekst(startwoord, aantal_woorden=20, temperature=1.0):
    """
    Genereer tekst woord voor woord.
    
    temperature: 
      - laag (0.1) = voorspelbaar, kiest bijna altijd het meest waarschijnlijke woord
      - hoog (2.0) = creatief/willekeurig, kiest vaker onverwachte woorden
    """
    huidig = startwoord.lower()
    resultaat = [huidig]
    
    for _ in range(aantal_woorden - 1):
        if huidig not in bigrammen:
            break
        
        # Haal de mogelijke volgende woorden op
        opties = bigrammen[huidig]
        volgende_woorden = list(opties.keys())
        tellingen = np.array(list(opties.values()), dtype=float)
        
        # Pas temperature toe
        tellingen = tellingen ** (1.0 / temperature)
        kansen = tellingen / tellingen.sum()
        
        # Kies het volgende woord
        huidig = np.random.choice(volgende_woorden, p=kansen)
        resultaat.append(huidig)
    
    return ' '.join(resultaat)

# Genereer tekst!
print("Gegenereerde tekst:")
print("-" * 50)
for i in range(5):
    print(f"  {genereer_tekst('the', 15)}")

## Stap 4: Het effect van Temperature 🌡️

Temperature is een instelling die ook ChatGPT en Claude gebruiken:
- **Lage temperature** (0.1): het model kiest bijna altijd het meest waarschijnlijke woord → voorspelbaar maar saai
- **Hoge temperature** (2.0): het model kiest willekeuriger → creatief maar soms onzin

In [ ]:
print("Effect van temperature op tekstgeneratie:")
print("=" * 60)

for temp in [0.1, 0.5, 1.0, 2.0]:
    print(f"\n🌡️ Temperature = {temp}:")
    for _ in range(3):
        print(f"  {genereer_tekst('the', 12, temperature=temp)}")

## Stap 5: Slimmer model — trigrammen

Ons bigram-model kijkt maar naar **1 woord** terug. Echte taalmodellen kijken naar duizenden woorden terug.

Laten we een iets slimmer model maken dat naar de **laatste 2 woorden** kijkt (trigrammen).

In [ ]:
# Bouw een trigram-model (kijkt naar 2 woorden)
trigrammen = defaultdict(Counter)

for i in range(len(woorden) - 2):
    context = (woorden[i], woorden[i + 1])  # De laatste 2 woorden
    volgend = woorden[i + 2]
    trigrammen[context][volgend] += 1

def genereer_trigram(startwoorden, aantal_woorden=20, temperature=1.0):
    w1, w2 = startwoorden[0].lower(), startwoorden[1].lower()
    resultaat = [w1, w2]
    
    for _ in range(aantal_woorden - 2):
        context = (w1, w2)
        if context not in trigrammen:
            break
        
        opties = trigrammen[context]
        volgende_woorden = list(opties.keys())
        tellingen = np.array(list(opties.values()), dtype=float)
        tellingen = tellingen ** (1.0 / temperature)
        kansen = tellingen / tellingen.sum()
        
        volgend = np.random.choice(volgende_woorden, p=kansen)
        resultaat.append(volgend)
        w1, w2 = w2, volgend
    
    return ' '.join(resultaat)

print("Bigram-model (kijkt 1 woord terug):")
for _ in range(3):
    print(f"  {genereer_tekst('the', 15)}")

print(f"\nTrigram-model (kijkt 2 woorden terug):")
for _ in range(3):
    print(f"  {genereer_trigram(['the', 'cat'], 15)}")

## Stap 6: Vergelijking met echte taalmodellen

Laten we de aanpak vergelijken:

In [ ]:
vergelijking = {
    'Eigenschap':           ['Ons bigram-model',     'Ons trigram-model',       'ChatGPT / Claude'],
    'Kijkt terug':          ['1 woord',              '2 woorden',               '~100.000+ tokens'],
    'Vocabulaire':          [f'{len(set(woorden))} woorden', f'{len(set(woorden))} woorden', '~100.000 tokens'],
    'Trainingsdata':        [f'{len(woorden)} woorden', f'{len(woorden)} woorden', 'Miljarden woorden'],
    'Begrijpt betekenis':   ['Nee',                  'Nee',                     'Ja (via embeddings)'],
    'Parameters':           ['~100',                 '~500',                    '~175 miljard (GPT-4)'],
}

import pandas as pd
df = pd.DataFrame(vergelijking)
df = df.set_index('Eigenschap')
df

### Het kernprincipe is hetzelfde!

Ondanks het enorme verschil in schaal, is het basisprincipe identiek:

> **Gegeven de voorgaande context, wat is het meest waarschijnlijke volgende woord?**

Het verschil zit in:
- Hoeveel context het model kan verwerken
- Dat echte modellen **embeddings** gebruiken (notebook 4) in plaats van losse woorden
- Dat echte modellen **transformers** gebruiken om relaties tussen alle woorden tegelijk te begrijpen
- De enorme hoeveelheid trainingsdata

## 🧪 Experimenteer zelf!

Pas de trainingstekst aan en kijk wat er gebeurt. Probeer bijvoorbeeld een tekst over een ander onderwerp.

In [ ]:
# Probeer met je eigen tekst!
eigen_tekst = """
The sun was shining bright. The sun made the flowers grow. The flowers were red and yellow.
The rain came and the flowers drank the water. The sun came back and the flowers smiled.
The wind blew the flowers gently. The flowers danced in the wind.
The sun set behind the mountains. The flowers closed their petals.
The moon rose and the stars appeared. The flowers slept until morning.
The sun rose again and the flowers opened their petals.
""".strip()

# Herbouw het model met de nieuwe tekst
eigen_woorden = eigen_tekst.lower().split()
eigen_bigrammen = defaultdict(Counter)
for i in range(len(eigen_woorden) - 1):
    eigen_bigrammen[eigen_woorden[i]][eigen_woorden[i + 1]] += 1

# Genereer
for _ in range(5):
    huidig = 'the'
    resultaat = [huidig]
    for _ in range(12):
        if huidig in eigen_bigrammen:
            opties = eigen_bigrammen[huidig]
            volgend = random.choices(list(opties.keys()), weights=list(opties.values()))[0]
            resultaat.append(volgend)
            huidig = volgend
        else:
            break
    print(' '.join(resultaat))